# Boltzmann Machines: A Comprehensive Introduction

## Overview
A **Boltzmann Machine (BM)** is a stochastic recurrent neural network inspired by statistical mechanics. Developed by Geoffrey Hinton and Terrence Sejnowski (1985), it serves as a **generative model** for learning complex probability distributions.

## Key Concepts

### 1. Architecture
- **Neurons**: Binary units (typically {0,1} or {-1,1})
- **Connections**: 
  - Symmetric weights (wᵢⱼ = wⱼᵢ)
  - No self-connections
- **Layers**:
  - **Visible units**: Represent observed data
  - **Hidden units**: Capture latent features

### 2. Energy Function
The energy of state (v,h) is defined as:


# Boltzmann Machines

## Mathematical Foundations

### Energy Function
The energy of a joint configuration $(v, h)$ is given by:

$$
E(v, h) = -\sum_i a_i v_i - \sum_j b_j h_j - \sum_{i,j} v_i w_{i,j} h_j
$$

Where:
- $v_i$: visible unit $i$
- $h_j$: hidden unit $j$ 
- $a_i$: bias for visible unit $i$
- $b_j$: bias for hidden unit $j$
- $w_{i,j}$: weight between units $i$ and $j$

### Probability Distribution
The probability of a configuration follows the Boltzmann distribution:

$$
P(v, h) = \frac{1}{Z} e^{-E(v, h)}
$$

Where $Z$ is the partition function:

$$
Z = \sum_{v,h} e^{-E(v, h)}
$$

### Conditional Probabilities
For an RBM, the conditional probabilities factorize:

$$
P(h|v) = \prod_j P(h_j|v) \\
P(v|h) = \prod_i P(v_i|h)
$$

With individual unit activations:

$$
P(h_j=1|v) = \sigma(b_j + \sum_i v_i w_{i,j}) \\
P(v_i=1|h) = \sigma(a_i + \sum_j h_j w_{i,j})
$$

Where $\sigma(x) = \frac{1}{1+e^{-x}}$ is the sigmoid function.

## Training Algorithm (Contrastive Divergence)

1. **Positive Phase**:
   Compute $\langle v_i h_j \rangle_{data}$ using $P(h|v^{(0)})$

2. **Negative Phase**:
   - Sample $h^{(0)} \sim P(h|v^{(0)})$
   - Sample $v^{(1)} \sim P(v|h^{(0)})$
   - Sample $h^{(1)} \sim P(h|v^{(1)})$
   Compute $\langle v_i h_j \rangle_{recon}$

3. **Weight Update**:
   $$
   \Delta w_{i,j} = \eta (\langle v_i h_j \rangle_{data} - \langle v_i h_j \rangle_{recon})
   $$

## Implementation Pseudocode

```python
def train_rbm(data, n_visible, n_hidden, learning_rate, epochs):
    # Initialize parameters
    W = random_normal([n_visible, n_hidden])
    a = zeros([n_visible])  # visible biases
    b = zeros([n_hidden])   # hidden biases
    
    for epoch in range(epochs):
        # Positive phase
        h0_prob = sigmoid(data @ W + b)
        h0_sample = binomial(h0_prob)
        
        # Negative phase
        v1_prob = sigmoid(h0_sample @ W.T + a)
        v1_sample = binomial(v1_prob)
        h1_prob = sigmoid(v1_sample @ W + b)
        
        # Update parameters
        grad_W = data.T @ h0_prob - v1_sample.T @ h1_prob
        grad_a = mean(data - v1_sample, axis=0)
        grad_b = mean(h0_prob - h1_prob, axis=0)
        
        W += learning_rate * grad_W
        a += learning_rate * grad_a
        b += learning_rate * grad_b

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import Binarizer

class RBM:
    def __init__(self, n_visible, n_hidden, learning_rate=0.1, epochs=100):
        self.n_visible = n_visible
        self.n_hidden = n_hidden
        self.lr = learning_rate
        self.epochs = epochs
        
        # Initialize weights and biases
        self.W = np.random.randn(n_visible, n_hidden) * 0.01
        self.a = np.zeros(n_visible)  # Visible layer bias
        self.b = np.zeros(n_hidden)   # Hidden layer bias
    
    def sigmoid(self, x):
        return 1 / (1 + np.exp(-x))
    
    def sample_hidden(self, v):
        # Sample hidden layer given visible layer
        h_prob = self.sigmoid(np.dot(v, self.W) + self.b)
        h_sample = np.random.binomial(1, h_prob)
        return h_prob, h_sample
    
    def sample_visible(self, h):
        # Sample visible layer given hidden layer
        v_prob = self.sigmoid(np.dot(h, self.W.T) + self.a)
        v_sample = np.random.binomial(1, v_prob)
        return v_prob, v_sample
    
    def train(self, X):
        errors = []
        for epoch in range(self.epochs):
            # Forward pass: compute hidden layer probabilities and sample
            h0_prob, h0_sample = self.sample_hidden(X)
            
            # Backward pass: reconstruct visible layer and sample hidden layer
            v1_prob, v1_sample = self.sample_visible(h0_sample)
            h1_prob, h1_sample = self.sample_hidden(v1_sample)
            
            # Update weights and biases using Contrastive Divergence (CD-1)
            positive_grad = np.dot(X.T, h0_prob)
            negative_grad = np.dot(v1_sample.T, h1_prob)
            
            self.W += self.lr * (positive_grad - negative_grad) / X.shape[0]
            self.a += self.lr * np.mean(X - v1_sample, axis=0)
            self.b += self.lr * np.mean(h0_prob - h1_prob, axis=0)
            
            # Calculate reconstruction error
            error = np.mean((X - v1_prob) ** 2)
            errors.append(error)
            
            if epoch % 10 == 0:
                print(f"Epoch {epoch}, Error: {error:.4f}")
        
        return errors
    
    def reconstruct(self, X):
        _, h = self.sample_hidden(X)
        _, v = self.sample_visible(h)
        return v

# Load MNIST dataset (binarized)
mnist = fetch_openml('mnist_784', version=1)
X = mnist.data.astype('float32') / 255.0
X = Binarizer(threshold=0.5).fit_transform(X)  # Binarize
X_train, X_test = train_test_split(X, test_size=0.2, random_state=42)

# Train RBM
rbm = RBM(n_visible=784, n_hidden=64, learning_rate=0.01, epochs=50)
errors = rbm.train(X_train)

# Visualize training error
plt.plot(errors)
plt.xlabel("Epoch")
plt.ylabel("Reconstruction Error")
plt.title("RBM Training Error")
plt.show()

# Test reconstruction
test_sample = X_test[:5]
reconstructed = rbm.reconstruct(test_sample)

# Visualize original vs. reconstructed images
fig, axes = plt.subplots(2, 5, figsize=(10, 4))
for i in range(5):
    axes[0, i].imshow(test_sample[i].reshape(28, 28), cmap='gray')
    axes[0, i].set_title("Original")
    axes[0, i].axis('off')
    axes[1, i].imshow(reconstructed[i].reshape(28, 28), cmap='gray')
    axes[1, i].set_title("Reconstructed")
    axes[1, i].axis('off')
plt.tight_layout()
plt.show()